## 1. 聚合函数
- 分组后得到的GroupBy对象可以使用聚合函数进行数据聚合，以下是一些常用的聚合函数：
![title](img/数据聚合.png)
- GroupBy对象还可以调用被分组对象里自带的任何函数（非聚合函数也可以）
- 自定义的聚合函数，只需使用`aggregate()`或`agg()`方法传入自定义函数，分组结果的各个切片调用自定义函数并返回运算结果
- 上表中的聚合函数，其实就是使用`agg()`方法调用聚合函数，被调用的聚合函数名称以字符串形式传入`agg()`，如`agg('mean')`
- 返回结果禁用索引：聚合运算后得到的结果默认都是以分组键作为索引，可在分组时使用`as_index=False`禁止将分组键作为索引而只作为普通的列，相当于对结果`reset_index()`

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'key1': list('ABBCBCAA'),
    'key2': list('YZXYZXYZ'),
    'data1': np.random.randint(100, size=8),
    'data2': np.random.randint(10, size=8)
})
df

,key1,key2,data1,data2
0,A,Y,4,7
1,B,Z,31,7
2,B,X,70,7
3,C,Y,80,3
4,B,Z,2,8
5,C,X,33,2
6,A,Y,39,6
7,A,Z,76,3


In [3]:
# 自定义函数进行聚合操作
def test(arr):
    if len(arr) > 1:
        return max(arr) - min(arr)
    else:
        return arr


df.groupby(['key1', 'key2']).agg(test)

data1  data2
key1 key2              
A    Y        35      1
     Z        76      3
B    X        70      7
     Z        29      1
C    X        33      2
     Y        80      3

In [4]:
# 以下语句相当于df.groupby(['key1','key2']).mean()
df.groupby(['key1', 'key2']).agg('mean')

data1  data2
key1 key2              
A    Y      21.5    6.5
     Z      76.0    3.0
B    X      70.0    7.0
     Z      16.5    7.5
C    X      33.0    2.0
     Y      80.0    3.0

In [5]:
# 分组时可以禁止将分组键作为索引
df.groupby(['key1', 'key2'],as_index=False).agg('mean')

,key1,key2,data1,data2
0,A,Y,21.5,6.5
1,A,Z,76.0,3.0
2,B,X,70.0,7.0
3,B,Z,16.5,7.5
4,C,X,33.0,2.0
5,C,Y,80.0,3.0


## 2.  应用多个聚合函数
如果需要对数据中不同的列分别应用不同的聚合函数，可以使用`agg()`来实现
- **对所有列应用多个聚合函数：**  
将多个函数的函数名以数组形式传入`agg()`，返回结果中的列将会以函数名来命名
- 若需要自定义返回的列名，可以传入一组元组列表，元组元素分别为自定义名称和函数名
- **对每个列应用不同聚合函数：**  
以字典形式传入列名和函数名，其中列名作为key，函数名作为value，多个函数的话由函数名组成列表，若要返回自定义列名则使用元组，`{'列名':[('自定义列名','函数名'),'函数名']}`
- 只有在列上应用了多个函数时，才会返回层次化数据

In [6]:
# 同时应用多种聚合函数，注意已有的聚合函数名以字符串形式传入，列名默认为函数名
df.groupby(['key1', 'key2']).agg(['mean', 'count', test])

data1            data2           
           mean count test  mean count test
key1 key2                                  
A    Y     21.5     2   35   6.5     2    1
     Z     76.0     1   76   3.0     1    3
B    X     70.0     1   70   7.0     1    7
     Z     16.5     2   29   7.5     2    1
C    X     33.0     1   33   2.0     1    2
     Y     80.0     1   80   3.0     1    3

In [7]:
# 应用多种聚合函数，并自定义列名
df.groupby(['key1', 'key2']).agg([('平均数', 'mean'), ('计数', 'count')])

data1    data2   
            平均数 计数   平均数 计数
key1 key2                  
A    Y     21.5  2   6.5  2
     Z     76.0  1   3.0  1
B    X     70.0  1   7.0  1
     Z     16.5  2   7.5  2
C    X     33.0  1   2.0  1
     Y     80.0  1   3.0  1

In [8]:
# 对每个列应用不同聚合函数
df.groupby(['key1', 'key2']).agg({
    'data1': [('最小', 'min'), ('计数', 'count')],
    'data2': ['mean', test]
})

data1    data2     
             最小 计数  mean test
key1 key2                    
A    Y        4  2   6.5    1
     Z       76  1   3.0    3
B    X       70  1   7.0    7
     Z        2  2   7.5    1
C    X       33  1   2.0    2
     Y       80  1   3.0    3

## 3. apply()方法
`apply(func)` 对已经被拆分成多个片段的GroupBy对象，将函数func应用到每个片段上，最后将得到的结果再组合到一起，如果函数func还需要其他参数，将这些参数放在函数名后一起传入。  
默认情况下分组键会和原索引共同构成运算后结果的索引，可以使用`group_keys=False`禁止分组键作为索引
**`apply()`和`agg()`区别：**  
- `apply()` 是将从表格中拆分出来的子表（DataFrame或者Series）应用到传入的函数上
- `agg()` 是将表格的一列数据应用到传入的函数上

In [9]:
df

,key1,key2,data1,data2
0,A,Y,4,7
1,B,Z,31,7
2,B,X,70,7
3,C,Y,80,3
4,B,Z,2,8
5,C,X,33,2
6,A,Y,39,6
7,A,Z,76,3


In [10]:
# 建立一个函数，将传入的数据按照指定列进行排序
def func_sort(df,columns='data1'):
    return df.sort_values(columns)

# 使用apply传入函数名，并在函数名后传入参数
df.groupby(['key1','key2']).apply(func_sort,columns=['data1','data2'])

key1 key2  data1  data2
key1 key2                          
A    Y    0    A    Y      4      7
          6    A    Y     39      6
     Z    7    A    Z     76      3
B    X    2    B    X     70      7
     Z    4    B    Z      2      8
          1    B    Z     31      7
C    X    5    C    X     33      2
     Y    3    C    Y     80      3

In [11]:
# 上述方法可以看作是将GroupBy对象中每个元素的数据部分应用func_sort()函数，最后再将结果组合起来
for x,y in df.groupby(['key1','key2']):
    print(x)
    print(func_sort(y,columns=['data1','data2'])) # 对每个数据部分应用函数

('A', 'Y')
  key1 key2  data1  data2
0    A    Y      4      7
6    A    Y     39      6
('A', 'Z')
  key1 key2  data1  data2
7    A    Z     76      3
('B', 'X')
  key1 key2  data1  data2
2    B    X     70      7
('B', 'Z')
  key1 key2  data1  data2
4    B    Z      2      8
1    B    Z     31      7
('C', 'X')
  key1 key2  data1  data2
5    C    X     33      2
('C', 'Y')
  key1 key2  data1  data2
3    C    Y     80      3


In [12]:
# 禁止分组键作为索引
df.groupby(['key1','key2'],group_keys=False).apply(func_sort,columns=['data1','data2'])

,key1,key2,data1,data2
0,A,Y,4,7
6,A,Y,39,6
7,A,Z,76,3
2,B,X,70,7
4,B,Z,2,8
1,B,Z,31,7
5,C,X,33,2
3,C,Y,80,3


In [13]:
# apply对于Series同样适用
f=lambda x:x.sort_values()
df['data1'].groupby(list('AABBABAA')).apply(f)

A  4     2
   0     4
   1    31
   6    39
   7    76
B  5    33
   2    70
   3    80
Name: data1, dtype: int32